In [293]:
# SCRIPT TO MERGE FED AND TREASURY DATA

# Libaries 
from cmath import nan
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
matplotlib.use('Qt5Agg')
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json
import urllib.request

from datetime import date, timedelta

# Link FRED Data https://fredaccount.stlouisfed.org/
from fredapi import Fred #Reference for commands: https://mortada.net/python-api-for-fred.html
#FRED search syntax
#fred.search('Liabilities: deposits with F.R. Banks').T

sdt = '2022-07-01'
edt = '2022-08-31'

# Link Treasury.gov data
baseurl = 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/dts/dts_table_1' #dts_table_1 is Operating Cash Balance
params = '?fields=record_date,account_type,open_today_bal,close_today_bal&sort=record_date&page[number]=1&page[size]=500'
format = '&format=json'
fedsdt = '&filter=record_date:gte:'+sdt
fullurl = baseurl+params+fedsdt+format
#print(fullurl)
################################################## SECTION IMPORTING FED DATA ######################################################
fred = Fred(api_key='f35f22064bbf8cb1bd7e23fa10cadaab')
df = {}
df['WALCL'] = fred.get_series('WALCL', observation_start=sdt, observation_end=edt) #Assets: Total Assets: Total Assets (Less Eliminations from Consolidation): Wednesday Level
#df['TGA'] = fred.get_series('WDTGAL', observation_start=sdt, observation_end=edt) #Liabilities and Capital: Liabilities: Deposits with F.R. Banks, Other Than Reserve Balances: U.S. Treasury, General Account: Wednesday Level 
#df['TGA'] = fred.get_series('WTREGEN', observation_start=sdt, observation_end=edt) * 1000 #Liabilities and Capital: Liabilities: Deposits with F.R. Banks, Other Than Reserve Balances: U.S. Treasury, General Account: Week Average 
df['REPO'] = fred.get_series('RRPONTTLD', observation_start=sdt, observation_end=edt) * 1000 #Overnight Reverse Repurchase Agreements: Total Securities Sold by the Federal Reserve in the Temporary Open Market Operations (RRPONTTLD)
#df['REPOT'] = fred.get_series('RRPONTSYD', observation_start=sdt, observation_end=edt) * 1000 #Overnight Reverse Repurchase Agreements: Treasury Securities Sold by the Federal Reserve in the Temporary Open Market Operations (RRPONTSYD)
# As of 7/27/2022, there was no difference between RRPONTTLD and RRPONTSYD

#df = pd.DataFrame(df,columns = ['dt','WALCL','TGA','REPO'])
df = pd.DataFrame(df)
df.index.name = 'dt'
intmethod = 'ffill' # Choose interpolation method: ‘backfill’, ‘bfill’, ‘pad’, ‘ffill’, None
df['WALCL'] = df['WALCL'].fillna(method=intmethod)

#df['LESSTGA'] = df['WALCL'] - df['TGA'] # Fed balance sheet without backing out Repos
#df['LIQ'] = df['WALCL'] - (df['TGA'] + df['REPO'])
df['SPX'] = fred.get_series('SP500', observation_start=sdt, observation_end=edt)

df = df[df.REPO.notnull()] #Show only days where Repo is reported (most commonly reported of these stats)

#df['SPX'] = df['SPX'].shift(-2) #Shift SPX two weeks for delayed effect

################################################## SECTION IMPORTING TREASURY DATA ######################################################

contents = urllib.request.urlopen(fullurl).read()

dft = json.loads(contents.decode())
dft2 = []
for record in dft['data']:
    if record['account_type'] == 'Treasury General Account (TGA) Closing Balance': #Post-April 2022
        dft2.append([record['record_date'], record['open_today_bal']])
    if record['account_type'] == 'Treasury General Account (TGA)': #Pre-April 2022
        dft2.append([record['record_date'], record['close_today_bal']])

dft2 = pd.DataFrame(dft2,columns = ['dt','TGA'])
#dft2['dt'].str.replace(' ','') # remove whitespace
#dft2['TGA'].str.replace(' ','') # remove whitespace
#dft2['TGA'] = df['TGA'].fillna(method=intmethod) #Fill any blanks for join process
dft2['TGA'] = dft2['TGA'].astype('float64')
dft2['dt'] = dft2['dt'].astype('datetime64')
dft2.set_index('dt',inplace=True)


#df.info()
#dft2.info()
dfmerge = df.join(dft2, on='dt',how='outer',lsuffix='_F',rsuffix='_T',sort=True)
dfmerge['TGA'] = dfmerge['TGA'].fillna(method='ffill') #Fill any blanks for join process
dfmerge['LESSTGA'] = dfmerge['WALCL'] - dfmerge['TGA'] # Fed balance sheet without backing out Repos
dfmerge['LIQ'] = dfmerge['WALCL'] - (dfmerge['TGA'] + dfmerge['REPO'])
#dfmerge.info()


col1, col2 = "LIQ", "SPX"
corr = dfmerge[col1].corr(dfmerge[col2])
print ("Since",sdt,",","Correlation between", col1, "and", col2, "is:", round(corr, 4))

corr = dfmerge[col1].corr(dfmerge[col2].shift(-1))
print ("Since",sdt, ",","Correlation between", col1, "and", col2, "is:", round(corr, 4), " using 1d shift")
corr = dfmerge[col1].corr(dfmerge[col2].shift(-2))
print ("Since",sdt, ",","Correlation between", col1, "and", col2, "is:", round(corr, 4), " using 2d shift")
corr = dfmerge[col1].corr(dfmerge[col2].shift(-3))
print ("Since",sdt, ",","Correlation between", col1, "and", col2, "is:", round(corr, 4), " using 3d shift")
corr = dfmerge[col1].corr(dfmerge[col2].shift(-4))
print ("Since",sdt, ",","Correlation between", col1, "and", col2, "is:", round(corr, 4), " using 4d shift")

corr = dfmerge[col1].corr(dfmerge[col2].shift(-5))
print ("Since",sdt, ",","Correlation between", col1, "and", col2, "is:", round(corr, 4), " using 1 week shift")
corr = dfmerge[col1].corr(dfmerge[col2].shift(-10))
print ("Since",sdt, ",","Correlation between", col1, "and", col2, "is:", round(corr, 4), " using 2 week shift")
corr = dfmerge[col1].corr(dfmerge[col2].shift(-15))
print ("Since",sdt, ",","Correlation between", col1, "and", col2, "is:", round(corr, 4), " using 3 week shift")
corr = dfmerge[col1].corr(dfmerge[col2].shift(-20))
print ("Since",sdt, ",","Correlation between", col1, "and", col2, "is:", round(corr, 4), " using 4 week shift")

dfmerge = dfmerge[['WALCL','TGA','REPO','LESSTGA','LIQ','SPX']] # reordering columns to my preference
dfmerge.tail(10)



Since 2022-07-01 , Correlation between LIQ and SPX is: 0.2128
Since 2022-07-01 , Correlation between LIQ and SPX is: 0.1376  using 1d shift
Since 2022-07-01 , Correlation between LIQ and SPX is: 0.2411  using 2d shift
Since 2022-07-01 , Correlation between LIQ and SPX is: 0.2049  using 3d shift
Since 2022-07-01 , Correlation between LIQ and SPX is: 0.1757  using 4d shift
Since 2022-07-01 , Correlation between LIQ and SPX is: 0.1309  using 1 week shift
Since 2022-07-01 , Correlation between LIQ and SPX is: -0.0351  using 2 week shift
Since 2022-07-01 , Correlation between LIQ and SPX is: 0.6734  using 3 week shift
Since 2022-07-01 , Correlation between LIQ and SPX is: -0.8403  using 4 week shift


,WALCL,TGA,REPO,LESSTGA,LIQ,SPX
dt,,,,,,
2022-07-28,8890004.0,597017.0,2239883.0,8292987.0,6053104.0,4072.43
2022-07-29,8890004.0,619273.0,2300200.0,8270731.0,5970531.0,4130.29
2022-08-01,8890004.0,551023.0,2161885.0,8338981.0,6177096.0,4118.63
2022-08-02,8890004.0,586370.0,2156013.0,8303634.0,6147621.0,4091.19
2022-08-03,8874620.0,566577.0,2182238.0,8308043.0,6125805.0,4155.17
2022-08-04,8874620.0,554402.0,2191546.0,8320218.0,6128672.0,4151.94
2022-08-05,8874620.0,548849.0,2194927.0,8325771.0,6130844.0,4145.19
2022-08-08,8874620.0,555513.0,2195692.0,8319107.0,6123415.0,4140.06
2022-08-09,8874620.0,583255.0,2186568.0,8291365.0,6104797.0,4122.47
